### 언어모델링

In [64]:
text="이 여름 다시 한번 설레고 싶다 그 여름 틀어줘"

In [65]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import SimpleRNN

In [66]:
train_X=[[0.1,4.2,1.5,1.1,2.8],[1.0,3.1,2.5,0.7,1.1],[0.3,2.1,1.5,2.1,0.1],[2.2,1.4,0.5,0.9,1.1]]
print(np.shape(train_X))

(4, 5)


In [67]:
train_X=[[[0.1,4.2,1.5,1.1,2.8],[1.0,3.1,2.5,0.7,1.1],[0.3,2.1,1.5,2.1,0.1],[2.2,1.4,0.5,0.9,1.1]]]
train_X=np.array(train_X,dtype=np.float32)
print(train_X)
print(train_X.shape)

[[[0.1 4.2 1.5 1.1 2.8]
  [1.  3.1 2.5 0.7 1.1]
  [0.3 2.1 1.5 2.1 0.1]
  [2.2 1.4 0.5 0.9 1.1]]]
(1, 4, 5)


In [68]:
rnn=SimpleRNN(3,return_sequences=True,return_state=True)

rnn(train_X)
print(rnn(train_X))

[<tf.Tensor: shape=(1, 4, 3), dtype=float32, numpy=
array([[[-0.97886175, -0.99964297, -0.94503814],
        [-0.2532577 , -0.99935764, -0.43845037],
        [ 0.843766  , -0.97704816,  0.7032845 ],
        [-0.53209364, -0.8566222 ,  0.05405154]]], dtype=float32)>, <tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[-0.53209364, -0.8566222 ,  0.05405154]], dtype=float32)>]


In [69]:
hidden_states,last_states=rnn(train_X)

print(train_X,train_X.shape)
print("hidden\n",hidden_states,hidden_states.shape)
print("last\n",last_states,last_states.shape)

[[[0.1 4.2 1.5 1.1 2.8]
  [1.  3.1 2.5 0.7 1.1]
  [0.3 2.1 1.5 2.1 0.1]
  [2.2 1.4 0.5 0.9 1.1]]] (1, 4, 5)
hidden
 tf.Tensor(
[[[-0.97886175 -0.99964297 -0.94503814]
  [-0.2532577  -0.99935764 -0.43845037]
  [ 0.843766   -0.97704816  0.7032845 ]
  [-0.53209364 -0.8566222   0.05405154]]], shape=(1, 4, 3), dtype=float32) (1, 4, 3)
last
 tf.Tensor([[-0.53209364 -0.8566222   0.05405154]], shape=(1, 3), dtype=float32) (1, 3)


In [70]:
from tensorflow.keras.layers import Dense,Embedding

rnn=SimpleRNN(3,return_sequences=True, return_state=True)
rnn=Dense(5,activation='softmax')
print(rnn(train_X))

tf.Tensor(
[[[0.08923786 0.09444843 0.32081667 0.13934524 0.3561518 ]
  [0.20763043 0.1117044  0.06167179 0.22886741 0.39012593]
  [0.2411448  0.16068406 0.0567423  0.18251108 0.35891774]
  [0.10585092 0.0925808  0.02641185 0.1351575  0.6399989 ]]], shape=(1, 4, 5), dtype=float32)


In [71]:
from keras_preprocessing.text import Tokenizer

t=Tokenizer()
t.fit_on_texts([text])

encoded=t.texts_to_sequences([text])[0]

In [72]:
t.texts_to_sequences([text])

[[2, 1, 3, 4, 5, 6, 7, 1, 8]]

In [73]:
t.texts_to_sequences([text])[0]

[2, 1, 3, 4, 5, 6, 7, 1, 8]

In [74]:
vocab_size=len(t.word_index)+1
print(vocab_size)

9


In [75]:
print(t.word_index)

{'여름': 1, '이': 2, '다시': 3, '한번': 4, '설레고': 5, '싶다': 6, '그': 7, '틀어줘': 8}


In [76]:
sequences=list()

for c in range(1,len(encoded)):
    sequence=encoded[c-1:c+1]
    sequences.append(sequence)
    
print(len(sequences))

8


In [77]:
print(sequences)

[[2, 1], [1, 3], [3, 4], [4, 5], [5, 6], [6, 7], [7, 1], [1, 8]]


In [78]:
X,y=zip(*sequences)

X=np.array(X)

y=np.array(y)

In [79]:
from tensorflow.keras.utils import to_categorical
y=to_categorical(y,num_classes=vocab_size)
print(y)

[[0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [80]:
from tensorflow.keras.models import Sequential
model=Sequential()
model.add(Embedding(vocab_size,3,input_length=1))

model.add(SimpleRNN(9))

model.add(Dense(vocab_size,activation='softmax'))

In [81]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(X,y,epochs=500)

Epoch 1/500
1/1 [==============================] - 1s 1s/step - loss: 2.1969 - accuracy: 0.1250
Epoch 2/500
1/1 [==============================] - 0s 2ms/step - loss: 2.1952 - accuracy: 0.1250
Epoch 3/500
1/1 [==============================] - 0s 3ms/step - loss: 2.1934 - accuracy: 0.1250
Epoch 4/500
1/1 [==============================] - 0s 3ms/step - loss: 2.1917 - accuracy: 0.1250
Epoch 5/500
1/1 [==============================] - 0s 4ms/step - loss: 2.1900 - accuracy: 0.1250
Epoch 6/500
1/1 [==============================] - 0s 4ms/step - loss: 2.1883 - accuracy: 0.1250
Epoch 7/500
1/1 [==============================] - 0s 2ms/step - loss: 2.1865 - accuracy: 0.2500
Epoch 8/500
1/1 [==============================] - 0s 3ms/step - loss: 2.1848 - accuracy: 0.2500
Epoch 9/500
1/1 [==============================] - 0s 4ms/step - loss: 2.1831 - accuracy: 0.2500
Epoch 10/500
1/1 [==============================] - 0s 4ms/step - loss: 2.1813 - accuracy: 0.2500
Epoch 11/500
1/1 [============

1/1 [==============================] - 0s 5ms/step - loss: 1.9981 - accuracy: 0.5000
Epoch 85/500
1/1 [==============================] - ETA: 0s - loss: 1.9947 - accuracy: 0.50 - 0s 3ms/step - loss: 1.9947 - accuracy: 0.5000
Epoch 86/500
1/1 [==============================] - 0s 4ms/step - loss: 1.9912 - accuracy: 0.5000
Epoch 87/500
1/1 [==============================] - 0s 3ms/step - loss: 1.9876 - accuracy: 0.5000
Epoch 88/500
1/1 [==============================] - 0s 5ms/step - loss: 1.9840 - accuracy: 0.5000
Epoch 89/500
1/1 [==============================] - 0s 5ms/step - loss: 1.9804 - accuracy: 0.5000
Epoch 90/500
1/1 [==============================] - 0s 3ms/step - loss: 1.9768 - accuracy: 0.5000
Epoch 91/500
1/1 [==============================] - 0s 6ms/step - loss: 1.9731 - accuracy: 0.5000
Epoch 92/500
1/1 [==============================] - 0s 6ms/step - loss: 1.9694 - accuracy: 0.5000
Epoch 93/500
1/1 [==============================] - 0s 4ms/step - loss: 1.9657 - accuracy

1/1 [==============================] - 0s 8ms/step - loss: 1.6175 - accuracy: 0.6250
Epoch 167/500
1/1 [==============================] - 0s 3ms/step - loss: 1.6118 - accuracy: 0.6250
Epoch 168/500
1/1 [==============================] - 0s 5ms/step - loss: 1.6062 - accuracy: 0.6250
Epoch 169/500
1/1 [==============================] - 0s 7ms/step - loss: 1.6005 - accuracy: 0.6250
Epoch 170/500
1/1 [==============================] - 0s 4ms/step - loss: 1.5948 - accuracy: 0.6250
Epoch 171/500
1/1 [==============================] - 0s 4ms/step - loss: 1.5890 - accuracy: 0.6250
Epoch 172/500
1/1 [==============================] - 0s 5ms/step - loss: 1.5833 - accuracy: 0.6250
Epoch 173/500
1/1 [==============================] - 0s 9ms/step - loss: 1.5776 - accuracy: 0.6250
Epoch 174/500
1/1 [==============================] - 0s 6ms/step - loss: 1.5718 - accuracy: 0.6250
Epoch 175/500
1/1 [==============================] - 0s 4ms/step - loss: 1.5660 - accuracy: 0.6250
Epoch 176/500
1/1 [=====

1/1 [==============================] - 0s 4ms/step - loss: 1.1284 - accuracy: 0.8750
Epoch 249/500
1/1 [==============================] - 0s 4ms/step - loss: 1.1226 - accuracy: 0.8750
Epoch 250/500
1/1 [==============================] - 0s 5ms/step - loss: 1.1168 - accuracy: 0.8750
Epoch 251/500
1/1 [==============================] - 0s 6ms/step - loss: 1.1111 - accuracy: 0.8750
Epoch 252/500
1/1 [==============================] - 0s 4ms/step - loss: 1.1053 - accuracy: 0.8750
Epoch 253/500
1/1 [==============================] - 0s 3ms/step - loss: 1.0996 - accuracy: 0.8750
Epoch 254/500
1/1 [==============================] - 0s 4ms/step - loss: 1.0939 - accuracy: 0.8750
Epoch 255/500
1/1 [==============================] - 0s 3ms/step - loss: 1.0883 - accuracy: 0.8750
Epoch 256/500
1/1 [==============================] - 0s 7ms/step - loss: 1.0826 - accuracy: 0.8750
Epoch 257/500
1/1 [==============================] - 0s 4ms/step - loss: 1.0770 - accuracy: 0.8750
Epoch 258/500
1/1 [=====

1/1 [==============================] - 0s 6ms/step - loss: 0.7353 - accuracy: 0.8750
Epoch 331/500
1/1 [==============================] - 0s 2ms/step - loss: 0.7316 - accuracy: 0.8750
Epoch 332/500
1/1 [==============================] - 0s 3ms/step - loss: 0.7280 - accuracy: 0.8750
Epoch 333/500
1/1 [==============================] - 0s 3ms/step - loss: 0.7244 - accuracy: 0.8750
Epoch 334/500
1/1 [==============================] - 0s 3ms/step - loss: 0.7208 - accuracy: 0.8750
Epoch 335/500
1/1 [==============================] - 0s 7ms/step - loss: 0.7173 - accuracy: 0.8750
Epoch 336/500
1/1 [==============================] - 0s 4ms/step - loss: 0.7137 - accuracy: 0.8750
Epoch 337/500
1/1 [==============================] - 0s 8ms/step - loss: 0.7102 - accuracy: 0.8750
Epoch 338/500
1/1 [==============================] - 0s 4ms/step - loss: 0.7068 - accuracy: 0.8750
Epoch 339/500
1/1 [==============================] - 0s 3ms/step - loss: 0.7033 - accuracy: 0.8750
Epoch 340/500
1/1 [=====

1/1 [==============================] - 0s 3ms/step - loss: 0.5097 - accuracy: 0.8750
Epoch 413/500
1/1 [==============================] - 0s 12ms/step - loss: 0.5077 - accuracy: 0.8750
Epoch 414/500
1/1 [==============================] - 0s 4ms/step - loss: 0.5057 - accuracy: 0.8750
Epoch 415/500
1/1 [==============================] - 0s 3ms/step - loss: 0.5038 - accuracy: 0.8750
Epoch 416/500
1/1 [==============================] - 0s 12ms/step - loss: 0.5018 - accuracy: 0.8750
Epoch 417/500
1/1 [==============================] - 0s 4ms/step - loss: 0.4998 - accuracy: 0.8750
Epoch 418/500
1/1 [==============================] - 0s 4ms/step - loss: 0.4979 - accuracy: 0.8750
Epoch 419/500
1/1 [==============================] - 0s 9ms/step - loss: 0.4960 - accuracy: 0.8750
Epoch 420/500
1/1 [==============================] - 0s 3ms/step - loss: 0.4941 - accuracy: 0.8750
Epoch 421/500
1/1 [==============================] - 0s 4ms/step - loss: 0.4922 - accuracy: 0.8750
Epoch 422/500
1/1 [===

1/1 [==============================] - 0s 4ms/step - loss: 0.3827 - accuracy: 0.8750
Epoch 495/500
1/1 [==============================] - 0s 4ms/step - loss: 0.3816 - accuracy: 0.8750
Epoch 496/500
1/1 [==============================] - 0s 4ms/step - loss: 0.3804 - accuracy: 0.8750
Epoch 497/500
1/1 [==============================] - 0s 7ms/step - loss: 0.3793 - accuracy: 0.8750
Epoch 498/500
1/1 [==============================] - 0s 3ms/step - loss: 0.3781 - accuracy: 0.8750
Epoch 499/500
1/1 [==============================] - 0s 9ms/step - loss: 0.3770 - accuracy: 0.8750
Epoch 500/500
1/1 [==============================] - 0s 4ms/step - loss: 0.3759 - accuracy: 0.8750


In [82]:
t=Tokenizer()
t.fit_on_texts([text])

encoded=t.texts_to_sequences([text])[0]

In [83]:
t.texts_to_sequences([text])

[[2, 1, 3, 4, 5, 6, 7, 1, 8]]

In [84]:
t.texts_to_sequences([text])[0]

[2, 1, 3, 4, 5, 6, 7, 1, 8]

In [85]:
print(t.word_index.items())

dict_items([('여름', 1), ('이', 2), ('다시', 3), ('한번', 4), ('설레고', 5), ('싶다', 6), ('그', 7), ('틀어줘', 8)])


In [86]:
def predict_next_word(model,t,current_word):
    encoded=t.texts_to_sequences([current_word])[0]
    encoded=np.array(encoded)
    result=model.predict_classes(encoded,verbose=0)
    
    for word,index in t.word_index.items():
        if index==result:
            return word

In [87]:
print(predict_next_word(model,t,'여름'))

다시


In [88]:
print(predict_next_word(model,t,'다시'))

한번


In [89]:
def sentence_generation(model,t,current_word,n):
    init_word=current_word
    sentence=''
    
    for _ in range(n):
        encoded=t.texts_to_sequences([current_word])[0]
        encoded=np.array(encoded)
        result=model.predict_classes(encoded,verbose=0)
        
        for word,index in t.word_index.items():
            if index==result:
                break
        current_word=word
        sentence=sentence+' '+word
    sentence=init_word+sentence
    return sentence

In [90]:
print(sentence_generation(model,t,'다시',6))

다시 한번 설레고 싶다 그 여름 다시


In [91]:
print(sentence_generation(model,t,'여름',6))

여름 다시 한번 설레고 싶다 그 여름


In [92]:
print(sentence_generation(model,t,'여름',10))

여름 다시 한번 설레고 싶다 그 여름 다시 한번 설레고 싶다


In [93]:
text="""경마장에 있는 말이 뛰고 있다. 그의 말이 법이다. 가는 말이 고와야 오는 말이 곱다."""

In [94]:
from keras_preprocessing.text import Tokenizer

t=Tokenizer()
t.fit_on_texts([text])

encoded=t.texts_to_sequences([text])[0]

In [95]:
vocab_size=len(t.word_index)+1
print(vocab_size)

12


In [96]:
print(t.word_index)

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [97]:
sequences=list()

for line in text.split('.'):
    encoded=t.texts_to_sequences([line])[0]
    for i in range(1,len(encoded)):
        sequence=encoded[:i+1]
        sequences.append(sequence)
    
print(len(sequences))

11


In [98]:
print(sequences)

[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


In [99]:
print(max(len(l) for l in sequences))

6


In [100]:
from keras.preprocessing.sequence import pad_sequences

sequences=pad_sequences(sequences,maxlen=6,padding='pre')
print(sequences)

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


In [101]:
import numpy as np
sequences=np.array(sequences)
X=sequences[:,:-1]
y=sequences[:,-1]

In [103]:
print(X)
print(y)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]
[ 3  1  4  5  1  7  1  9 10  1 11]


In [104]:
y=to_categorical(y,num_classes=vocab_size)
print(y)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [106]:
model=Sequential()
model.add(Embedding(vocab_size,10,input_length=5))

model.add(SimpleRNN(32))

model.add(Dense(vocab_size,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(X,y,epochs=200)

Epoch 1/200
1/1 [==============================] - 1s 1s/step - loss: 2.4587 - accuracy: 0.1818
Epoch 2/200
1/1 [==============================] - 0s 4ms/step - loss: 2.4445 - accuracy: 0.2727
Epoch 3/200
1/1 [==============================] - 0s 3ms/step - loss: 2.4298 - accuracy: 0.3636
Epoch 4/200
1/1 [==============================] - 0s 3ms/step - loss: 2.4147 - accuracy: 0.3636
Epoch 5/200
1/1 [==============================] - 0s 4ms/step - loss: 2.3991 - accuracy: 0.3636
Epoch 6/200
1/1 [==============================] - 0s 5ms/step - loss: 2.3830 - accuracy: 0.3636
Epoch 7/200
1/1 [==============================] - 0s 3ms/step - loss: 2.3663 - accuracy: 0.3636
Epoch 8/200
1/1 [==============================] - 0s 7ms/step - loss: 2.3489 - accuracy: 0.4545
Epoch 9/200
1/1 [==============================] - 0s 3ms/step - loss: 2.3309 - accuracy: 0.4545
Epoch 10/200
1/1 [==============================] - 0s 8ms/step - loss: 2.3122 - accuracy: 0.4545
Epoch 11/200
1/1 [============

1/1 [==============================] - 0s 6ms/step - loss: 0.8872 - accuracy: 0.8182
Epoch 85/200
1/1 [==============================] - 0s 6ms/step - loss: 0.8695 - accuracy: 0.8182
Epoch 86/200
1/1 [==============================] - 0s 5ms/step - loss: 0.8520 - accuracy: 0.8182
Epoch 87/200
1/1 [==============================] - 0s 6ms/step - loss: 0.8346 - accuracy: 0.8182
Epoch 88/200
1/1 [==============================] - 0s 4ms/step - loss: 0.8174 - accuracy: 0.9091
Epoch 89/200
1/1 [==============================] - 0s 8ms/step - loss: 0.8004 - accuracy: 0.9091
Epoch 90/200
1/1 [==============================] - 0s 5ms/step - loss: 0.7835 - accuracy: 0.9091
Epoch 91/200
1/1 [==============================] - 0s 4ms/step - loss: 0.7669 - accuracy: 0.9091
Epoch 92/200
1/1 [==============================] - 0s 8ms/step - loss: 0.7505 - accuracy: 0.9091
Epoch 93/200
1/1 [==============================] - 0s 4ms/step - loss: 0.7343 - accuracy: 0.9091
Epoch 94/200
1/1 [===============

1/1 [==============================] - 0s 8ms/step - loss: 0.1682 - accuracy: 1.0000
Epoch 168/200
1/1 [==============================] - 0s 4ms/step - loss: 0.1652 - accuracy: 1.0000
Epoch 169/200
1/1 [==============================] - 0s 5ms/step - loss: 0.1622 - accuracy: 1.0000
Epoch 170/200
1/1 [==============================] - 0s 11ms/step - loss: 0.1592 - accuracy: 1.0000
Epoch 171/200
1/1 [==============================] - 0s 7ms/step - loss: 0.1563 - accuracy: 1.0000
Epoch 172/200
1/1 [==============================] - 0s 4ms/step - loss: 0.1534 - accuracy: 1.0000
Epoch 173/200
1/1 [==============================] - 0s 7ms/step - loss: 0.1506 - accuracy: 1.0000
Epoch 174/200
1/1 [==============================] - 0s 4ms/step - loss: 0.1479 - accuracy: 1.0000
Epoch 175/200
1/1 [==============================] - 0s 5ms/step - loss: 0.1452 - accuracy: 1.0000
Epoch 176/200
1/1 [==============================] - 0s 5ms/step - loss: 0.1425 - accuracy: 1.0000
Epoch 177/200
1/1 [====

In [107]:
def sentence_generation(model,t,current_word,n):
    init_word=current_word
    sentence=''
    
    for _ in range(n):
        encoded=t.texts_to_sequences([current_word])[0]
        encoded=pad_sequences([encoded],maxlen=5,padding='pre')
        result=model.predict_classes(encoded,verbose=0)
        
        for word,index in t.word_index.items():
            if index==result:
                break
        current_word=current_word+' '+word
        sentence=sentence+' '+word
    sentence=init_word+sentence
    return sentence

In [108]:
print(sentence_generation(model,t,'경마장에',4))

경마장에 있는 말이 뛰고 있다


In [109]:
print(sentence_generation(model,t,'그의',2))

그의 말이 법이다


In [110]:
print(sentence_generation(model,t,'가는',5))

가는 말이 고와야 오는 말이 곱다


In [111]:
print(sentence_generation(model,t,'경마장에',7))

경마장에 있는 말이 뛰고 있다 뛰고 있다 말이
